In [1]:
import pandas as pd
import json
import os
from tqdm import tqdm
import pickle

gather a list of failed video ids
 - api fail
 - no video detail
create two df pickled files holding all successful video details and comments
 - videoID as keys
 - some videos disabled comments so n_video in video details will be more than n_video in comment file

In [2]:
# read in api failure list 
fp = '../data/raw/api_data/failed_lookups.txt'

with open(fp) as f:
    failed_dict = json.load(f)
    
failed_dict.keys()

dict_keys(['failed'])

In [3]:
failed_ids = set([lst[0] for lst in failed_dict['failed']]) #273

In [4]:
# with open('../data/processed/failed_ids.pkl', 'wb') as f:
#     pickle.dump(failed_ids, f)

In [41]:
def get_videoDetails(data, report = False):
    '''return a dataframe of all video details features from Youtube API pull and a set of failed id'''

    # for every yt8M id, check if videoDetail is available
    # if not, check if it's in failure list, if not, add to failure list
    # if it is, append to buffer list 
    
    buffer = []
    failed_ids = set()
    for idx in data.keys():
        if data[idx]['details'] == 'Invalid lookup.':
            failed_ids.add(idx) 
        elif len(data[idx]['details']['items']) == 0:
            failed_ids.add(idx)
        else:
            data[idx]['details']['items'][0]['yt8M_id'] = idx
            buffer.append(data[idx]['details']['items'][0])
    
    # json_normalize buffer list into videoDetails df
    videoDets_df = pd.json_normalize(buffer, errors='ignore')
    
    # print stats for batch
    if report:
        print(f'n of videos successful: {videoDets_df.shape[0]}')
        print(f'n of videos failed: {len(failed_ids)}')
    
    return videoDets_df, failed_ids
    
def get_videoComments(data, report=False):
    '''return a dataframe of all comments for videos from Youtube API pull'''
    
    no_comment_count = 0
    buffer = []
    for idx in data.keys():
        if data[idx]['comments'] == 'Invalid lookup':
            no_comment_count += 1
        elif len(data[idx]['comments']['items']) == 0:
            no_comment_count += 1
        else:
            # normalize video comments 
            coms_df = pd.json_normalize(data[idx]['comments']['items'], errors='ignore')
            coms_df['yt8M_id'] = idx
            buffer.append(coms_df)
    
    # concatent list of comments dfs into all videoComments df
    videoComs_df = pd.concat(buffer,ignore_index=True)
    
    # print stats for batch
    if report:
        print(f'{no_comment_count = }')

    return videoComs_df

In [42]:
# read in batch1
batch_1 = '../data/raw/api_data/batch_2.txt'

with open(batch_1) as b1:
    b1_videos = json.load(b1)

In [10]:
# get videoDetails and Comments
vDetails_df, _ = get_videoDetails(b1_videos, report=True)
vDetails_df.head()

n of videos successful: 396
n of videos failed: 4


,kind,etag,id,yt8M_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,localizations.en-GB.description,localizations.fr.title,localizations.fr.description,recordingDetails.locationDescription,recordingDetails.location.latitude,recordingDetails.location.longitude,recordingDetails.location.altitude,localizations.es-419.title,localizations.es-419.description,contentDetails.regionRestriction.allowed
0,youtube#video,5p4pcvVBl8HTGF61u3PT950pt_o,5u1e7tSYDzw,540z,2011-10-15T02:49:57Z,UCjpXYy_qrMdFeCzRw_oglKg,Bitter cold bucks with Noel Feather,This is still one of the greatest kills I've e...,https://i.ytimg.com/vi/5u1e7tSYDzw/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,youtube#video,ujxGbxHy_NTCEzPLUFjR12OH0WY,9UcEWF20SOs,qWf4,2012-11-27T13:28:25Z,UCtINLFlbso3M7zAtFcIFDjw,Nikon Coolpix P510 MOVIE Auto FOCUS Test,Nikon Coolpix P510 MOVIE Auto FOCUS Test\n\nFo...,https://i.ytimg.com/vi/9UcEWF20SOs/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,youtube#video,sqKvxMxfRvuiFTW4AT27nj1-ECk,WEIH2eb7bWE,t8O7,2011-05-06T13:04:03Z,UCUet4kaRjCHOPwp18ClGoig,Schmalreh mit kapitalem Bock im Anhang,Bei Beobachtungen im Revier bekam ich diesen k...,https://i.ytimg.com/vi/WEIH2eb7bWE/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,youtube#video,dqE79nwSQSx04W8IxWiC85TnO-I,bJ37R31h9OI,paI2,2011-07-16T11:23:09Z,UC4sIM6RuGTYRRqzHDfx4hEA,Igelkott - European Hedgehog - Erinaceus europ...,Igelkottar på min gårdsplan som äter och drick...,https://i.ytimg.com/vi/bJ37R31h9OI/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,youtube#video,H_kyulNlU5LAkROMWKHEFeX0fB8,4JBcJ8SXnCw,IZmT,2014-12-24T15:00:01Z,UCgPYOLbcgxSBCAmlaihQyMg,Safari Highlights #309: 22 - 24 November 2014 ...,All the latest wildlife sightings highlights f...,https://i.ytimg.com/vi/4JBcJ8SXnCw/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
vComments_df = get_videoComments(b1_videos, report=True)
len(vComments_df)#.head()

no_comment_count = 113


3716

In [12]:
# with open('../data/processed/batch1_videoDetails.pkl', 'wb') as f:
#     pickle.dump(vDetails_df, f)
    
# with open('../data/processed/batch1_videoComments.pkl', 'wb') as f:
#     pickle.dump(vComments_df, f)    

In [30]:
# trial code for reading in all data within directory
input_dir = '../data/raw/api_data'
all_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.startswith('batch')]
all_files

['../data/raw/api_data/batch_2.txt',
 '../data/raw/api_data/batch_3.txt',
 '../data/raw/api_data/batch_1.txt',
 '../data/raw/api_data/batch_4.txt',
 '../data/raw/api_data/batch_5.txt',
 '../data/raw/api_data/batch_7.txt',
 '../data/raw/api_data/batch_6.txt',
 '../data/raw/api_data/batch_11.txt',
 '../data/raw/api_data/batch_8.txt',
 '../data/raw/api_data/batch_9.txt',
 '../data/raw/api_data/batch_10.txt']

In [35]:
sample_files = all_files
sample_files.sort()
sample_files

['../data/raw/api_data/batch_1.txt',
 '../data/raw/api_data/batch_10.txt',
 '../data/raw/api_data/batch_11.txt',
 '../data/raw/api_data/batch_2.txt',
 '../data/raw/api_data/batch_3.txt',
 '../data/raw/api_data/batch_4.txt',
 '../data/raw/api_data/batch_5.txt',
 '../data/raw/api_data/batch_6.txt',
 '../data/raw/api_data/batch_7.txt',
 '../data/raw/api_data/batch_8.txt',
 '../data/raw/api_data/batch_9.txt']

In [37]:
failure_ids = set()
vDets_buffer = []
vComs_buffer = []

for f in tqdm(sample_files):
    print(f)
    with open(f) as f:
        json_f = json.load(f)
    
    vDets_df, fail_ids = get_videoDetails(json_f, report=True)
    vDets_buffer.append(vDets_df)
    failed_ids.update(fail_ids)
        
    vComs_df = get_videoComments(json_f, report=True)
    vComs_buffer.append(vComs_df)
        
    print('\n')
vDets = pd.concat(vDets_buffer,ignore_index=True)
vComs = pd.concat(vComs_buffer,ignore_index=True)    

 18%|████████                                    | 2/11 [00:00<00:00, 10.48it/s]

../data/raw/api_data/batch_1.txt
n of videos successful: 393
n of videos failed: 8
no_comment_count = 106


../data/raw/api_data/batch_10.txt
n of videos successful: 390
n of videos failed: 10
no_comment_count = 134


../data/raw/api_data/batch_11.txt


 45%|████████████████████                        | 5/11 [00:00<00:00, 12.89it/s]

n of videos successful: 233
n of videos failed: 9
no_comment_count = 67


../data/raw/api_data/batch_2.txt
n of videos successful: 396
n of videos failed: 4
no_comment_count = 113


../data/raw/api_data/batch_3.txt
n of videos successful: 394
n of videos failed: 6
no_comment_count = 112


../data/raw/api_data/batch_4.txt


 64%|████████████████████████████                | 7/11 [00:00<00:00, 14.57it/s]

n of videos successful: 394
n of videos failed: 6
no_comment_count = 113


../data/raw/api_data/batch_5.txt
n of videos successful: 392
n of videos failed: 8
no_comment_count = 112


../data/raw/api_data/batch_6.txt
n of videos successful: 393
n of videos failed: 7
no_comment_count = 121


../data/raw/api_data/batch_7.txt


100%|███████████████████████████████████████████| 11/11 [00:00<00:00, 13.79it/s]

n of videos successful: 393
n of videos failed: 7
no_comment_count = 108


../data/raw/api_data/batch_8.txt
n of videos successful: 394
n of videos failed: 6
no_comment_count = 108


../data/raw/api_data/batch_9.txt
n of videos successful: 390
n of videos failed: 10
no_comment_count = 117




In [24]:
vDets.sample(15)

,kind,etag,id,yt8M_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,localizations.no.title,localizations.no.description,localizations.af.title,localizations.af.description,localizations.zu.title,localizations.zu.description,localizations.zh-Hans.title,localizations.zh-Hans.description,localizations.pt.title,localizations.pt.description
2809,youtube#video,qKbWzCSlc-bkH0DqIBH847dXGKI,j1TE1kGHteU,URpz,2013-09-06T04:44:39Z,UCymPMyABkYA1dB2DbG_QUEw,Suppressed 300 Blackout Takes a Hog,Trying out a new Remington 700 SPS Tactical 30...,https://i.ytimg.com/vi/j1TE1kGHteU/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
667,youtube#video,koYUtVnss-BEpT2s131xpgucQFg,_chh9Q10aN0,JK4v,2007-04-22T09:16:17Z,UCb0-3scqpK9QpNBCJ-nX30Q,Ceci's Florida Panthers,As promised I have remade this video tribute f...,https://i.ytimg.com/vi/_chh9Q10aN0/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1534,youtube#video,nR9ny3X5a7YLMcaG0-VLVYkJ33g,6XMoBbbiGmA,cHjj,2015-05-28T05:36:48Z,UC_YY06Jk-2UD6hbcGEdPIOQ,EL OSO ANDINO,"El Oso Andino u Oso de Anteojos, especie prote...",https://i.ytimg.com/vi/6XMoBbbiGmA/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1820,youtube#video,x8lbnU3vtonjo5Dcz9qhc54k3ec,B158nyG2HQU,9BOp,2011-03-07T11:55:59Z,UCc7BlflVm5D4k0NjyYlAgiA,"'Terrible', podenco andaluz y zorro. Fox Hunting.","Un rastro de unos trescientos metros, a un zor...",https://i.ytimg.com/vi/B158nyG2HQU/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3307,youtube#video,bP6WXRE4qDROvmVTdmascSBF-YI,P0X0LJWX4NY,0vTC,2015-05-15T12:20:34Z,UCnavGPxEijftXneFxk28srA,"Amazing whale encounter, Slash rocks for eleph...","Nearly 100 pangolins freed in Indonesia, Slash...",https://i.ytimg.com/vi/P0X0LJWX4NY/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2426,youtube#video,Jqc1J9GkDk5rmMn0VB1HVD-sh0c,ZYBKHIpZOW4,Z6bN,2014-05-12T09:27:48Z,UCBEfEIeu4nzZCYDt5NbqmSg,LOBO IBERICO: Burros para proteger al ganado d...,Noticia: Programa Agrosfera TVE2 - 01/03/2014\...,https://i.ytimg.com/vi/ZYBKHIpZOW4/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448,youtube#video,2vsfAGdDF6Glvhkc6C0fwO8-TIQ,hS4A4lCg2hk,Tfjh,2008-08-04T00:04:28Z,UCejU-9OVycuTjW-6VKZbmNg,The Temple Of The Tigers,"Bangkok, Thailand -- The bass rattle beneath t...",https://i.ytimg.com/vi/hS4A4lCg2hk/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873,youtube#video,pI1guum2jSGqAj37Q_uhWTF4IBc,Tn-HZ95K_wQ,IWBa,2007-06-24T00:58:49Z,UCzKmDhlGHstVmT5_-87aUDQ,Elk Fight - King of the Harem Yellowstone Nati...,Bull elk fighting in the Madison River in Yell...,https://i.ytimg.com/vi/Tn-HZ95K_wQ/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2178,youtube#video,OKX3JNLjbjXT48yw9UO7iwLDYME,FR5paiKmWYY,9sFJ,2007-05-15T06:43:08Z,UC5_4kv_-3WVERPfi4JHxmBA,A Visit to Sydney Aquarium & Animal World,I had the opportunity to walk around these fas...,https://i.ytimg.com/vi/FR5paiKmWYY/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3270,youtube#video,ny3BRFdrqMe9zyIa2LRniTI3nGA,CUbFOd5UjW0,WoI7,2010-10-02T20:01:05Z,UCSxsntnc-Rpwc329MBLnHvw,les dragons de Komodo.2mpg.mpg,varans geants sur l'ile de Rinca en Indonesie,https://i.ytimg.com/vi/CUbFOd5UjW0/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
vComs.sample(10)

,kind,etag,id,yt8M_id,snippet.videoId,snippet.topLevelComment.kind,snippet.topLevelComment.etag,snippet.topLevelComment.id,snippet.topLevelComment.snippet.videoId,snippet.topLevelComment.snippet.textDisplay,...,snippet.topLevelComment.snippet.authorChannelId.value,snippet.topLevelComment.snippet.canRate,snippet.topLevelComment.snippet.viewerRating,snippet.topLevelComment.snippet.likeCount,snippet.topLevelComment.snippet.publishedAt,snippet.topLevelComment.snippet.updatedAt,snippet.canReply,snippet.totalReplyCount,snippet.isPublic,snippet.topLevelComment.snippet.moderationStatus
277,youtube#commentThread,Y18kqDEzlwJADJD0PbrGEkLsK0I,UgicFtHfhhAcsHgCoAEC,Qmr9,5ku38HP1QOE,youtube#comment,qQYTw_EViRtHd_PXSbOdoWwca_M,UgicFtHfhhAcsHgCoAEC,5ku38HP1QOE,That is absolutely brilliant. WOW,...,UCe1J1GgtWk5_y3i5i9cfk5Q,True,none,3,2015-04-02T07:20:29Z,2015-04-02T07:20:29Z,True,0,True,NaN
221,youtube#commentThread,Zn-XcRm-J-2LaqgB6IuUDtSvjaM,UgzOIHS5tmzoBMO98614AaABAg,e5XO,feNQK5hE5B4,youtube#comment,HEclrk6CBupt_YQmrU23S_U7504,UgzOIHS5tmzoBMO98614AaABAg,feNQK5hE5B4,2:50 that Vulture King be coming in like some ...,...,UCfHNoNEuoDYXBD8AkvjGDLQ,True,none,20,2021-08-02T05:15:00Z,2021-08-02T05:15:00Z,True,0,True,NaN
31,youtube#commentThread,KUJDkAqYRQh9Da_01OtSguImI3c,UgyYLSWb7T2Ib-hlgCl4AaABAg,3MK0,wD-crkl8U0Y,youtube#comment,oaH1sxXwBFPbuYKFYkIflZ7YFVY,UgyYLSWb7T2Ib-hlgCl4AaABAg,wD-crkl8U0Y,มนุษย์ชอบเอาเปรียบสัตว์มีปืนเป็นเครืองทุ่นแรงย...,...,UCKF4SN-3a24Z5Y-G56XBLHQ,True,none,0,2017-12-29T01:46:35Z,2017-12-29T01:46:35Z,True,0,True,NaN
253,youtube#commentThread,6lLH8te7SHx0dclhG_6_DeeEdI8,UgyWoWlA-kuJB4nSCRZ4AaABAg,ylP0,ok6UAxugYpA,youtube#comment,iIk1p-jGJbkUa0CVftQc45x8Pig,UgyWoWlA-kuJB4nSCRZ4AaABAg,ok6UAxugYpA,Saw David perform all three of his shows at th...,...,UCINKMFczP61iZ5iDy0daU1A,True,none,0,2020-06-04T18:57:40Z,2020-06-04T18:57:40Z,True,0,True,NaN
20,youtube#commentThread,cf6JD7nRV2Kif0j8k8WtPitabE0,Ugwpp7dK-N_Icmp3x214AaABAg,N18Y,gbm9KuYnQ6I,youtube#comment,-MuxSs8r6HEydKub3tGBmrbj6Go,Ugwpp7dK-N_Icmp3x214AaABAg,gbm9KuYnQ6I,사실 아프리카에 가서 봉사를 하는 것이 쉽지 않은 일인 것은 사실입니다. 비단 지금...,...,UCFTZfRq7yngxHdcz6nuXTQA,True,none,0,2021-05-05T08:42:10Z,2021-05-05T08:42:10Z,True,0,True,NaN
263,youtube#commentThread,BoSxMLQOMpnx49BqPsEFbaEjMBc,UgxjWEDV6CeGIDsvYhB4AaABAg,SZFE,gy5KGtnRhcE,youtube#comment,5hAKEii5aeBjL9H5enDAAJTFNDY,UgxjWEDV6CeGIDsvYhB4AaABAg,gy5KGtnRhcE,"Nice clip ,,,,the bucks are feeding up already...",...,UCWI4h87V-zvZbLAibh8aLig,True,none,0,2010-11-16T16:01:12Z,2010-11-16T16:01:12Z,True,0,True,NaN
112,youtube#commentThread,0rJvmTTO5JP4GxHjUXMyx0DQb7U,Ugyx26H2HrVBoZONmxN4AaABAg,BZv2,oW2MrwAtXsk,youtube#comment,jsNHRFM4xBo0Zh9bkCz_t6Uo6kI,Ugyx26H2HrVBoZONmxN4AaABAg,oW2MrwAtXsk,No offence taken what-so-ever! I've always bel...,...,UCdwJ7rLKfgT_ydZENWjM9qg,True,none,0,2007-07-13T19:59:14Z,2007-07-13T19:59:14Z,True,0,True,NaN
57,youtube#commentThread,mINOeQKONx0VV17P506dz0NQIOI,Ugwt-a5lm-DLPKxZrY14AaABAg,Pkmb,KmvhdNwu1PU,youtube#comment,m52G6Ga-Gz6PSfoOJ7sIb2JqQVQ,Ugwt-a5lm-DLPKxZrY14AaABAg,KmvhdNwu1PU,very cuteeeeeeeeee !!! :),...,UCS-g5QGJmav2gg4SNRQD7kg,True,none,0,2011-05-29T08:46:06Z,2011-05-29T08:46:06Z,True,0,True,NaN
167,youtube#commentThread,yRlV02Hezbik6gQ6qauDM2wL98c,UgyfpI-IK9B0YF6x2v54AaABAg,FnSQ,F9JneMNmlrc,youtube#comment,QzIxA8x3uvrnTIGY5mSjtLLfk78,UgyfpI-IK9B0YF6x2v54AaABAg,F9JneMNmlrc,Looks like spring new babies new life,...,UCwsWJkTlBnXepX1dO4ndPvQ,True,none,1,2019-02-01T00:11:10Z,2019-02-01T00:11:10Z,True,0,True,likelySpam
65,youtube#commentThread,QYgW6KnB--wzZ3eU_MBPnLRdWBk,UggXv_vjd73_XngCoAEC,MfNZ,0Xcd6c5BhEY,youtube#comment,zYP57JhKwlBaKQgoCIMpdB3EC_o,UggXv_vjd73_XngCoAEC,0Xcd6c5BhEY,"If you've heard of Pamana, the Philippine Eagl...",...,UC7PeVPpCjqC8DPp8pJiYOrA,True,none,36,2015-08-25T22:58:38Z,2015-08-25T22:58:38Z,True,11,True,NaN


In [5]:
# read in test_batch 
test_batch = '../data/processed/test_batch.txt'

with open(test_batch) as tb:
    test_files = json.load(tb)

test_files.keys()

dict_keys(['HTft', 'VX3C', 'j2Er', '3HOO', '6nfW', 'NX6e', 'ljAU', 'zmk6', 'qX7a', '3Z4A', 'kugn', '2D9o', 'Tvnd', 'A9Jp', 'f3CR'])

In [7]:
test_df, _ = get_videoDetails(test_files, report=True)
test_df.head()

n of videos successful: 15
n of videos failed: 0


,kind,etag,id,yt8M_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,status.embeddable,status.publicStatsViewable,status.madeForKids,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,player.embedHtml,topicDetails.topicCategories,recordingDetails.recordingDate
0,youtube#video,qw6REl1mcyQq7UoW1-Dkm1yCrOY,130S1TN3P0M,HTft,2014-11-05T07:49:51Z,UCh_KdtEAc7qwyeXyVuuYPUA,Obserwator 22: Jesień na tamie / Autumn on a ...,To już pożegnanie z tamą bobrową. Jesień zawit...,https://i.ytimg.com/vi/130S1TN3P0M/default.jpg,120,...,True,True,False,2415,19,0,3,"<iframe width=""480"" height=""360"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
1,youtube#video,hBay_xFV_egFO5I_5lVHS8xcSi4,iHpgw8WN6Bg,VX3C,2011-11-01T06:43:23Z,UCgBA-PogWxh44p-lKZW9EVA,bébés ocelots et otocyons au Domaine des Fauves,"les bébés ocelots âgés de 7 semaines, arrivée ...",https://i.ytimg.com/vi/iHpgw8WN6Bg/default.jpg,120,...,True,True,False,5582,27,0,1,"<iframe width=""480"" height=""360"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,2011-10-31T00:00:00Z
2,youtube#video,R0zGKh5qNMofm0IBhZHvobnWkqM,XrX9NBMUOzw,j2Er,2006-08-05T18:57:44Z,UC4ONXuixSMf5o9O67A4gzhg,Two Giraffe Calves born at the San Francisco Zoo,Follow the zoo: \r\nhttp://twitter.com/sfzoo\r...,https://i.ytimg.com/vi/XrX9NBMUOzw/default.jpg,120,...,True,True,False,289627,245,0,59,"<iframe width=""480"" height=""360"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,2006-05-09T00:00:00Z
3,youtube#video,XY93dMczSpUYyP18PTAnGLAW7Ws,qwL5HKti-dw,3HOO,2009-12-03T14:46:09Z,UCWc9ziDlznO7v7cjzuT-Pqw,Fauna salbatica bucovineana in fotografi - Buk...,#WildBucovina Natura si fauna sălbatică din Bu...,https://i.ytimg.com/vi/qwL5HKti-dw/default.jpg,120,...,True,True,False,4456,23,0,5,"<iframe width=""480"" height=""360"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Hobby, https://...",NaN
4,youtube#video,NcYca-t2DZKiXg5HhMYTuQeGBoI,wPJy9D_kzVU,6nfW,2010-07-13T06:23:18Z,UCtCCW9be1rUw_ig4UWsOSNg,Wyoming Archery Mule Deer Hunt With Joe Thomas...,,https://i.ytimg.com/vi/wPJy9D_kzVU/default.jpg,120,...,True,True,False,1338,2,0,0,"<iframe width=""480"" height=""270"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Hobby, https://...",NaN


In [8]:
test_df = get_videoComments(test_files, report=True)
test_df.head()

no_comment_count = 4


,kind,etag,id,yt8M_id,snippet.videoId,snippet.topLevelComment.kind,snippet.topLevelComment.etag,snippet.topLevelComment.id,snippet.topLevelComment.snippet.videoId,snippet.topLevelComment.snippet.textDisplay,...,snippet.topLevelComment.snippet.authorChannelUrl,snippet.topLevelComment.snippet.authorChannelId.value,snippet.topLevelComment.snippet.canRate,snippet.topLevelComment.snippet.viewerRating,snippet.topLevelComment.snippet.likeCount,snippet.topLevelComment.snippet.publishedAt,snippet.topLevelComment.snippet.updatedAt,snippet.canReply,snippet.totalReplyCount,snippet.isPublic
0,youtube#commentThread,EJSyRFx1dhqppMPt15RFmeUpnnM,UgjMGXaQw47fE3gCoAEC,HTft,130S1TN3P0M,youtube#comment,H5M32lmblp9WMe-eHLfBPKr_GrE,UgjMGXaQw47fE3gCoAEC,130S1TN3P0M,2:56 jeśli się nie mylę to pardwa:) Występuje ...,...,http://www.youtube.com/channel/UCJrxiBRv5IHv1U...,UCJrxiBRv5IHv1UJfKULEW7Q,True,none,0,2017-04-26T09:48:13Z,2017-04-26T09:48:13Z,True,2,True
1,youtube#commentThread,Oh-NTWh_vFipTc-3BKBTVz0LTEs,UgxLCQCLsca5zhKol2h4AaABAg,VX3C,iHpgw8WN6Bg,youtube#comment,cTsnqv7CcJvgJ0pznTYOoEt7vwY,UgxLCQCLsca5zhKol2h4AaABAg,iHpgw8WN6Bg,<3<3<3<3<3<3\nTrop ADORABLES!,...,http://www.youtube.com/channel/UCkHRUDRkrJPUXQ...,UCkHRUDRkrJPUXQfar1hChOg,True,none,0,2012-03-12T14:27:16Z,2012-03-12T14:27:16Z,True,0,True
2,youtube#commentThread,KHGlSgVVApoRzxY9_nMqbYTXWtE,UgySkF4KV4tbia-wGAF4AaABAg,j2Er,XrX9NBMUOzw,youtube#comment,pMET6ORoowLzZavBMrKFS1dpZPg,UgySkF4KV4tbia-wGAF4AaABAg,XrX9NBMUOzw,I loved how momma tapped at the calf to get th...,...,http://www.youtube.com/channel/UCc3BnE0KTjJzf6...,UCc3BnE0KTjJzf621gdN8L-A,True,none,0,2008-02-18T05:44:42Z,2008-02-18T05:44:42Z,True,0,True
3,youtube#commentThread,sXQJ7lTCShzrGvoqvf-p7OkJKA4,UgyccrM8fmN9z7gpArF4AaABAg,3HOO,qwL5HKti-dw,youtube#comment,WO9MkbKm5F3-XdRnDzJ3tnVPeDQ,UgyccrM8fmN9z7gpArF4AaABAg,qwL5HKti-dw,"Dacă ți-a plăcut, dă un ""LIKE"" și un ""SHARE"", ...",...,http://www.youtube.com/channel/UCWc9ziDlznO7v7...,UCWc9ziDlznO7v7cjzuT-Pqw,True,none,0,2022-07-08T12:56:11Z,2022-07-08T12:56:11Z,True,0,True
4,youtube#commentThread,Bm7jY1bz8gU1S1tt2BVeF26uE1Y,UgwQDLyNGIReuoaqvS94AaABAg,NX6e,S-MjHWCF_Cw,youtube#comment,pwQZ8vjRu669O7VLPjr4HvACbQE,UgwQDLyNGIReuoaqvS94AaABAg,S-MjHWCF_Cw,One day I'll decide to harvest!!! Thanks Rober...,...,http://www.youtube.com/channel/UCu_YcCDBu_UCmX...,UCu_YcCDBu_UCmXf2nmpw3vg,True,none,0,2009-11-24T12:18:59Z,2009-11-24T12:18:59Z,True,0,True
